In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from laptrack import LapTrack
import ray

In [3]:
df = pd.read_csv('/tungstenfs/scratch/ggiorget/nessim/cohesin_live_cell_analysis/results/20230321_Rad21-Halo_SPT_1C5_NIPBL_LP4_40mW_75pM_30min_3_FullseqTIRF-Cy5-mCherryGFPWithSMB/detections_in_cell__20230321_Rad21-Halo_SPT_1C5_NIPBL_LP4_40mW_75pM_30min_3_FullseqTIRF-Cy5-mCherryGFPWithSMB__w1.csv')

In [4]:
ray.init(num_cpus=20)

lt = LapTrack(parallel_backend ="ray",track_cost_cutoff=3.0**2,track_dist_metric='sqeuclidean',
                  gap_closing_cost_cutoff=7.0**2,gap_closing_max_frame_count=4) # track_cost_cutoff and gap_closing_cutoff should be the squared maximum distance", 
track_df, _, _ = lt.predict_dataframe(df, ["y", "x"], only_coordinate_cols=False,validate_frame=False)
track_df = track_df.reset_index()
track_df = track_df[['x','y','frame','track_id']]
ray.shutdown()

2023-05-29 11:20:53,694	INFO worker.py:1625 -- Started a local Ray instance.


In [10]:
u, c = np.unique(track_df.track_id.values, return_counts=True)
dup = u[c > 1]
track_m = track_df[track_df.track_id.isin(dup)]

In [18]:
m = track_m.groupby('track_id').size() > 23

In [24]:
m[m == True]

track_id
0        True
11       True
23       True
32       True
34       True
         ... 
20458    True
20499    True
20765    True
20900    True
20955    True
Length: 2721, dtype: bool

In [6]:
track_df.to_csv('tracks_NIPBL_multiprocess.csv')

In [7]:
tracks_single = pd.read_csv('/tungstenfs/scratch/ggiorget/nessim/cohesin_live_cell_analysis/results/20230321_Rad21-Halo_SPT_1C5_NIPBL_LP4_40mW_75pM_30min_3_FullseqTIRF-Cy5-mCherryGFPWithSMB/tracks__20230321_Rad21-Halo_SPT_1C5_NIPBL_LP4_40mW_75pM_30min_3_FullseqTIRF-Cy5-mCherryGFPWithSMB__w1.csv')

In [13]:
tracks_single

,Unnamed: 0,x,y,frame,track_id,z,cell_id
0,0,448.543953,14.119227,0,0,0,33
1,1,400.012721,27.653642,0,1,0,45
2,2,333.535614,34.212896,0,2,0,46
3,3,394.680432,33.583905,0,3,0,45
4,4,327.758917,37.860785,0,4,0,46
...,...,...,...,...,...,...,...
229969,236174,226.301456,502.394431,180,24980,4,49
229970,236175,138.274350,505.788061,180,25088,4,2
229971,236176,289.366735,507.519770,180,24674,4,52
229972,236177,303.725059,508.849566,180,15600,4,52


In [12]:
track_m == tracks_single

ValueError: Can only compare identically-labeled DataFrame objects